In [1]:
import bezier
from tkinter import *
import numpy as np
from PIL import ImageGrab

In [21]:
from tkinter import *
import numpy as np
import re #regex
#back end libraries for loading/saving/modifying our data
import StrokeFit as sf
from FitDef import FitData
import FitDef as fd
from StrokeDef import StrokeType #needed for loading object properly
import StrokeDef as sd
import FormalImage as fi
import cv2 #allows resizing images
from PIL import Image, ImageTk #allows settting canvas image from array
import bezier
import colorsys

#helper function for coloring
def getColor(h,s,v): #get hex code from hsv
    rgb = tuple((np.array(colorsys.hsv_to_rgb(h,s,v))*255).astype(int)) #very ugly, goes from 0-1 rgb, to 255 valued
    return "#" + '%02x%02x%02x' % rgb

def getColors(num, s=.9, v=.5): #will evenly spread colors across hues
    hues = np.linspace(0,1,num+1)[:-1] #ignore the last since hue is cyclic
    colors = []
    for hue in hues:
        colors.append(getColor(hue,s,v))
    return colors

def betweenDist(coord, minV, maxV): #helper function for figuring out click distance between two lines
    if(minV > maxV):
        return betweenDist(coord, maxV, minV) #minV and maxV are in wrong order, swap them
    if(coord < minV):
        return minV - coord
    elif(coord > maxV):
        return maxV - coord
    else: #between
        return 0

#classes for each of their frames and corresponding events
class CharFrame():
    def __init__(self, app):
        self.frame = Frame(app.topFrame, bg=app.lightBlue)
        self.frame.pack(side='left',  fill='both', padx=app.padding,  pady=app.padding)
        #var for entries
        self.code = StringVar(value="4f4d")
        self.char = StringVar(value="\u4f4d")
        #content
        self.label = Label(self.frame, bg=app.lightBlue, text="Symbol:", font= ('Arial 22'), padx=15)
        self.label.pack(side='left')
        self.codeEntry = Entry(self.frame, text="", font= ('Arial 22'), width=8, textvariable=self.code)
        self.codeEntry.pack(side='left', padx=25)
        self.charEntry = Entry(self.frame, text="", font= ('Arial 22'), width=4, textvariable=self.char)
        self.charEntry.pack(side='left', padx=15)

        
class StrokeItem():
    def __init__(self, sf, currNum, bgColor, allowDelete=True): #sf be the StrokeFrame class, some stroke may not be deleted if they are auto generated.
        self.strokeChoice = StringVar(value=sf.strokeOptions[0]) #default option
        self.groupChoice = StringVar(value=sf.groupOptions[0])
        
        self.frame = Frame(sf.scrollFrame, bg=app.offWhite)
        self.frame.pack(side='top', fill='y', padx=10, pady=10)
        #content
        self.nameLabel = Label(self.frame, text="Stroke #" + str(currNum), bg=app.offWhite, font=('Arial 15'), fg="white")
        self.nameLabel.grid(sticky="w",row=0, column=0, columnspan=2)
        if(allowDelete):
            self.deleteButton = Button(self.frame, text="X", font=('Arial 14'), bg="#F34848")
            self.deleteButton.grid(sticky="w", row=0, column=4)
        self.typeLabel = Label(self.frame, text="Type:", bg=app.offWhite, font=('Arial 14'), fg="white")
        self.typeLabel.grid(row=1, column=0)
        self.typeOptions = OptionMenu(self.frame, self.strokeChoice, *sf.strokeOptions)
        self.typeOptions.config(width=8)
        self.typeOptions.grid(row=1, column=1)
        self.groupLabel = Label(self.frame, text="Group:", bg=app.offWhite, font=('Arial 14'), fg="white")
        self.groupLabel.grid(row=2, column=0)
        self.groupOptions = OptionMenu(self.frame, self.groupChoice, *sf.groupOptions)
        self.groupOptions.config(width=4)
        self.groupOptions.grid(row=2, column=1)
        self.detectButton = Button(self.frame, text="Detect", font=('Arial 14'))
        self.detectButton.grid(row=2, column=2)
        self.editButton = Button(self.frame, text="Edit", font=('Arial 14'))
        self.editButton.grid(row=2, column=3)
        
        self.editColor = getColor(.6, .5, .8) #otherwise white
        self.editMode = False
        
        self.deleted = False #used for keeping track of deleted strokes.
    def toggleEdit(self): #flip the editMode state
        if(self.deleted):
            return
        if(self.editMode):
            self.stopEdit()
        else:
            self.startEdit()
    def startEdit(self):
        if(self.deleted):
            return
        self.editMode = True
        self.editButton.configure(bg=self.editColor)
    def stopEdit(self):
        if(self.deleted):
            return
        self.editMode = False
        self.editButton.configure(bg="#F0F0F0") #default color
        
    def changeColor(self, color): #change the primary color of the frame
        self.frame.configure(bg=color)
        self.typeLabel.configure(bg=color)
        self.groupLabel.configure(bg=color)
        self.nameLabel.configure(bg=color)

class StrokeFrame():
    def __init__(self, app):
        self.white = app.offWhite #used for stroke item
        #code for the stroke frame in the bottom left
        self.frame = Frame(app.bottomFrame, bg=app.lightBlue)
        self.frame.pack(side='left',  fill='y', padx=app.padding, pady=app.padding)
        #weird configuration needed for scrollsing, needs a canvas, and frame inside
        self.canvas = Canvas(self.frame, bg=app.lightBlue)
        self.scrollFrame = Frame(self.canvas, bg=app.lightBlue)
        self.scroll = Scrollbar(self.frame, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scroll.set)
        self.scroll.pack(side="right", fill='y')
        self.canvas.pack(side="left", fill="both", expand=1) 
        #configuration for stroke items
        self.strokeList = [] #list of stroke items
        self.strokeOptions = [*str("㇀㇁㇂㇃㇄㇅㇆㇇㇈㇉㇊㇋㇌㇍㇎㇏㇐㇑㇒㇓㇔㇕㇖㇗㇘㇙㇚㇛㇜㇝㇞㇟㇠㇡㇢㇣")] #list where each char is an entry
        self.groupOptions = range(1,15) #list where each char is an entry
        #content
        self.newButton = Button(self.scrollFrame, text="New Stroke", padx=40, font=('Arial 18'))
        self.newButton.pack(side='bottom', fill='y', padx=10, pady=10)
        #events
        self.canvas.bind("<Configure>", self.onFrameConfigure)
        self.canvas.bind("<Configure>", self.onFrameConfigure)
        self.scrollFrame.bind_all("<MouseWheel>", self.mouseScroll)
    
    def addNewStrokeItem(self, delete=True): #delete chooses if the stroke is deleteable or not
        newItem = StrokeItem(self, len(self.strokeList)+1, self.white, allowDelete=delete) #StrokeFrame, strokeNum, backgroundColor
        self.strokeList.append(newItem)
        self.frame.update() #forces scrollbar to update
        self.onFrameConfigure(0) #redo the frame since we've added contents
        self.canvas.yview_moveto(1) #scroll to the bottom of the frame
        
    def onFrameConfigure(self, event): #note event isn't
        self.canvas.configure(scrollregion = self.canvas.bbox("all"))
        wrapFrame = self.canvas.create_window((0,0), window=self.scrollFrame, anchor="nw")
        self.canvas.itemconfigure(wrapFrame, width=self.canvas.winfo_width())
    
    def mouseScroll(self, event):
        #thanks: https://stackoverflow.com/questions/17355902/tkinter-binding-mousewheel-to-scrollbar
        self.canvas.yview_scroll(1,"units") if(event.delta < 0) else self.canvas.yview_scroll(-1,"units") # scroll up else scroll down
        
    def clearStrokeItems(self): #used to reset the stroke item list
        for ind in range(len(self.strokeList)):
            self.strokeList[ind].frame.destroy() #remove this stroke item from the ui
            self.strokeList[ind].deleted = True
        self.strokeList.clear() #delete the actual objects
        self.frame.update() #forces scrollbar to update
        self.onFrameConfigure(0) #allows scrollbar to be updated
        
    def adjustColoring(self): #adjust colors for all the stroke boxes
        count = len(self.strokeList)
        colors = getColors(count)
        for i in range(count):
            self.strokeList[i].changeColor(colors[i])
        
class PictureFrame():
    def __init__(self, app):
        self.frame = Frame(app.bottomFrame, bg=app.mainBlue)
        self.frame.pack(side='right', fill='both', expand=True, padx = app.padding,  pady = app.padding)
        #content
        self.canvasSize=100 #not correct initial size, but the resize event will fix this on app startup
        self.canvas = Canvas(self.frame, bg="white", width=self.canvasSize, height=self.canvasSize) #our drawing board
        self.canvas.pack(side="top")
        #events
        self.frame.bind('<Configure>', self.resize)
        
        self.fitCopy = None #used for redrawing on resizing, not an authoratative copy of the fitData
        self.strokeDict = None #used for redrawing on resizing, not an authoratative copy of the fitData
        self.charImage = None #background image object
        self.boxes = [] #stroke bounding boxes
        self.strokes = [] #stroke curve lines, each element is a list of the segments for that stroke
        self.edit = -1 #-1 means no stroke has edit priority, otherwise that index matches the stroke number
        self.eval = np.linspace(0.0, 1.0, 25) #used for plotting bezier curves, parametric values
        
    def resize(self,event):
        oldSize = self.canvasSize
        self.canvasSize = min(event.width, event.height)-10
        self.canvas.config(width=self.canvasSize, height=self.canvasSize)
        if(oldSize != self.canvasSize): #don't redraw unless new size
            self.setFromFits(self.fitCopy, self.strokeDict, setCopy=False)
        
    def setFromFits(self, fitData, strokeDict, setCopy=True): #set copy is if we update our local copy of the fit and strokeDict
        if(fitData is None or strokeDict is None): #this event was called before data was set, ignore
            return
        if(setCopy):
            self.fitCopy = fitData
            self.strokeDict = strokeDict
        self.boxes = [] #reset since we're redrawing
        self.strokes = []
        self.colors = [] #hex code of all the box colors
        self.canvas.delete('all') #remove all fits
        #load and set the char image
        arrayImg = 200 + 55*(1-fi.renderChar(fitData.character, size=1000, pad=.1)) #white maps to 255, black maps to 200, pad .1 is typical
        arrayImg = cv2.resize(arrayImg, dsize=(self.canvasSize, self.canvasSize), interpolation=cv2.INTER_NEAREST)
        self.img = ImageTk.PhotoImage(Image.fromarray(arrayImg)) #has to be saved to avoid being garbage collected
        self.charImage = self.canvas.create_image(0,0, image=self.img, anchor="nw")
        
        mappedFits = sf.mapFits(fitData.fits, fitData.size, [self.canvasSize, self.canvasSize], 0) #padding should be the same on both
        self.colors = getColors(len(fitData.fits), v=.8)
        for i in range(len(fitData.fits)):
            fit = mappedFits[i]
            self.boxes.append(self.canvas.create_rectangle(fit.x[0],fit.y[0],fit.x[1],fit.y[1], outline=self.colors[i], width=2, dash=(3,1)))
            
            curveLines = []
            stroke = sf.mapStroke(strokeDict[fitData.fits[i].name].arial, fit.x, fit.y)
            for seg in stroke:
                nodes = np.array(seg).transpose()
                curve = bezier.Curve(nodes, degree=len(seg)-1)
                curvePoints = curve.evaluate_multi(self.eval).transpose().tolist() #[x_value list, y_value list]
                curveLines.append(self.canvas.create_line(curvePoints, fill='#000000', width=2))
            self.strokes.append(curveLines)
        if(self.edit != -1): #adjust the edited stroke if need be
            self.editStroke(self.edit, True)
            
    def editStroke(self, ind, editMode):
        if(editMode):
            self.canvas.itemconfig(self.boxes[ind], dash=(), width=3)
        else:
            self.canvas.itemconfig(self.boxes[ind], dash=(3,1,3,1), width=2)
            
    def redrawStroke(self, ind, editMode):
        self.canvas.delete(self.boxes[ind]) #delete box and curve segments
        for curveId in self.strokes[ind]:
            self.canvas.delete(curveId)
        
        fit = sf.mapFits([self.fitCopy.fits[ind]], self.fitCopy.size, [self.canvasSize, self.canvasSize], 0)[0] #map only this fit for the bounding box
        self.boxes[ind] = self.canvas.create_rectangle(fit.x[0],fit.y[0],fit.x[1],fit.y[1], outline=self.colors[ind], width=2, dash=(3,1))
        self.editStroke(ind, editMode)
        
        curveLines = []
        stroke = sf.mapStroke(self.strokeDict[self.fitCopy.fits[ind].name].arial, fit.x, fit.y)
        for seg in stroke:
            nodes = np.array(seg).transpose()
            curve = bezier.Curve(nodes, degree=len(seg)-1)
            curvePoints = curve.evaluate_multi(self.eval).transpose().tolist() #[x_value list, y_value list]
            curveLines.append(self.canvas.create_line(curvePoints, fill='#000000', width=2))
        self.strokes[ind] = curveLines
            
        
class OptionsFrame():
    def __init__(self, app):
        self.frame = Frame(app.topFrame, bg=app.mainBlue)
        self.frame.pack(side = 'right',  fill = 'both',  padx = app.padding,  pady = app.padding,  expand=True)
        #content
        self.autoButton = Button(self.frame, text="Auto-Gen", font=('Arial 14'))
        self.autoButton.pack(side='left', padx=20, pady=10)
        self.loadButton = Button(self.frame, text="Load", font=('Arial 14'))
        self.loadButton.pack(side='left', padx=20, pady=10)
        self.exportButton = Button(self.frame, text="Save & Export", font=('Arial 14'))
        self.exportButton.pack(side='right', padx=20, pady=10)
        self.saveButton = Button(self.frame, text="Save", font=('Arial 14'))
        self.saveButton.pack(side='right', padx=20, pady=10)
        #events

class App:
    def run(self):
        self.root.mainloop() #run the app
    
    def __init__(self, width = 1200, height = 850, padding=10):
        #setup vars for colors and config
        self.width = width
        self.height = height
        self.padding = padding
        self.mainBlue = "#5A837B"
        self.lightBlue = "#B3CCC8"
        self.offWhite = "#CFE7E2"
        
        #setup initial app window
        self.root = Tk()
        self.root.geometry( str(width) + "x" + str(height) )
        self.root.configure(background=self.mainBlue)
        #top frame for options and character
        self.topFrame = Frame(self.root, bg=self.mainBlue)
        self.topFrame.pack(side='top', fill="both")
        #bottom frame for stroke editor and picture frame
        self.bottomFrame = Frame(self.root, bg=self.mainBlue)
        self.bottomFrame.pack(side='bottom', fill="both", expand=True)

        self.cf = CharFrame(self) #init the top left character frame
        self.of = OptionsFrame(self) #init the top right options frame
        self.sf = StrokeFrame(self) #init the bottom left stroke frame
        self.pf =  PictureFrame(self) #init the bottom right picture frame
        
        self.me = MiddleEvents(self) #register all the important events
        
#registers all events needed to communicate with the middle layer e.g. rendering images
class MiddleEvents:
    def __init__(self, app):
        self.app = app
        
        self.fitData = FitData()
        self.strokeDict = sd.loadStrokeDict()
        self.strokeNames = []
        for strokeType in self.strokeDict.values():
            self.strokeNames.append(strokeType.symbol + " " + strokeType.name) #e.g. D1 Vert |
        self.app.sf.strokeOptions = self.strokeNames #update drop down list for strokes
        
        self.app.of.autoButton.configure(command = self.autoClick)
        self.app.of.loadButton.configure(command = self.loadClick)
        self.app.of.saveButton.configure(command = self.saveClick)
        self.app.of.exportButton.configure(command = self.exportClick)
        
        self.app.pf.canvas.bind("<Button-1>", self.canvasClick)
        self.app.pf.canvas.bind("<ButtonRelease-1>", self.canvasClickRelease)
        self.app.pf.canvas.bind("<B1-Motion>", self.canvasDrag)
        
        self.app.cf.code.trace("w", self.codeChange)
        self.app.cf.char.trace("w", self.charChange)
        
        self.app.sf.newButton.configure(command = self.newStrokeAdded)
        
        self.codeUpdated = False #determines whether to use text from the code or char box to determine what character we're using.
        
        self.selectedEdge = -1 # 0 - left, 1 - right, 2 - top, 3 - down, -1 - none
    
    def codeChange(self, *args): #e.g. 45E2
        self.codeUpdated = True
        text = self.app.cf.code.get()
        filtered = "".join(re.findall("[a-f,A-F,\d]", text)) #only characters related to unicode are allowed
        if(len(filtered) > 6): #too long
            filtered = filtered[:6]
        self.app.cf.code.set(filtered)
        
    def charChange(self, *args): #character entered, one char max
        self.codeUpdated = False
        text = self.app.cf.char.get()
        if(len(text) > 1): #only one character is allowed
            self.app.cf.char.set(text[0])
        
    def getChar(self): #get the desired character (either from the code or char box), updates other to match
        if(self.codeUpdated):
            text = self.app.cf.codeEntry.get()
            character =  chr(int(text, 16)) #char from the unicode num in hex
            self.app.cf.char.set(character)
        else: #char box
            character = self.app.cf.charEntry.get()
            self.app.cf.code.set( hex(ord(character))[2:] )
        return character
    
    def autoClick(self):
        char = self.getChar()
        print("auto-gen:", char)
        #call middleware process character
        self.loadClick()
    def loadClick(self):
        char = self.getChar()
        tempData = fd.loadFits(char)
        if(tempData is None):
            print(f"{char} fit data not found!")
            return
        self.fitData = tempData #set, since data loaded properly
        self.app.sf.clearStrokeItems() #reset the stroke item list
        for i in range(len(self.fitData.fits)):
            fit = self.fitData.fits[i]
            self.app.sf.addNewStrokeItem()
            self.app.sf.strokeList[i]
            #change the properties of item to match the fit
            strokeType = self.strokeDict[fit.name]
            self.app.sf.strokeList[i].strokeChoice.set(strokeType.symbol + " " + strokeType.name)
            self.app.sf.strokeList[i].groupChoice.set(fit.group)
            #bind events
            self.app.sf.strokeList[i].strokeChoice.trace("w", lambda *args, ind=i, me=self: me.strokeOptionChange(ind))
            self.app.sf.strokeList[i].groupChoice.trace("w", lambda *args, ind=i, me=self: me.strokeGroupChange(ind))
            self.app.sf.strokeList[i].deleteButton.configure(command = lambda ind=i, me=self: me.strokeDelete(ind))
            self.app.sf.strokeList[i].detectButton.configure(command = lambda ind=i, me=self: me.strokeDetect(ind)) 
            self.app.sf.strokeList[i].editButton.configure(command = lambda ind=i, me=self: me.strokeEdit(ind))
        #set canvas objects and update
        self.app.pf.setFromFits(self.fitData, self.strokeDict)
        self.app.sf.adjustColoring()
        
    def saveClick(self):
        print("save")
        #convert canvas objects and save
    def exportClick(self):
        #save & and then export
        self.saveClick()
        print("export")
        
    
    #canvas events ----
    def canvasClick(self, event):
        if(self.app.pf.edit != -1): #a certain stroke is being edited
            #map x and y to proper position
            x = sf.mapCoord(event.x, self.app.pf.canvasSize, self.fitData.size[0], 0) #0 is the padding
            y = sf.mapCoord(event.y, self.app.pf.canvasSize, self.fitData.size[1], 0)
            #figure out which edge is closest
            fit = self.fitData.fits[self.app.pf.edit]
            leftDist  = abs(x-fit.x[0]) + betweenDist(y, fit.y[0], fit.y[1])
            rightDist = abs(x-fit.x[1]) + betweenDist(y, fit.y[0], fit.y[1])
            upDist    = abs(y-fit.y[0]) + betweenDist(x, fit.x[0], fit.x[1])
            downDist  = abs(y-fit.y[1]) + betweenDist(x, fit.x[0], fit.x[1])
            
            self.selectedEdge = -1
            currDist = 10e10
            minDist = 15 #furthest mouse can be away to count as a valid click
            if(leftDist < minDist):
                minDist = leftDist
                self.selectedEdge = 0
            if(rightDist < currDist and rightDist < minDist):
                minDist = rightDist
                self.selectedEdge = 1
            if(upDist < currDist and upDist < minDist):
                minDist = upDist
                self.selectedEdge = 2
            if(downDist < currDist and downDist < minDist):
                #no need to set min dist, this is the last one
                self.selectedEdge = 3
            
            
    def canvasDrag(self, event): #"<B1-Motion>"
        if(self.app.pf.edit != -1 and self.selectedEdge != -1): #a certain stroke is being edited and mouse clicked on an edge
            #map x and y to proper position
            x = sf.mapCoord(event.x, self.app.pf.canvasSize, self.fitData.size[0], 0) #0 is the padding
            y = sf.mapCoord(event.y, self.app.pf.canvasSize, self.fitData.size[1], 0)
            i = self.app.pf.edit #stroke editing index
            if(self.selectedEdge == 0): #left
                self.fitData.fits[i].x[0] = x
                self.app.pf.fitCopy.fits[i].x[0] = x
            elif(self.selectedEdge == 1): #right
                self.fitData.fits[i].x[1] = x
                self.app.pf.fitCopy.fits[i].x[1] = x
            elif(self.selectedEdge == 2): #up
                self.fitData.fits[i].y[0] = y
                self.app.pf.fitCopy.fits[i].y[0] = y
            else: #down
                self.fitData.fits[i].y[1] = y
                self.app.pf.fitCopy.fits[i].y[1] = y
            self.app.pf.redrawStroke(i, True) #second arg is edit mode
            
    def canvasClickRelease(self, event):
        return #doesn't do anything currently
        #print("click release, coords:", event.x, event.y)
    #maybe a right click event needed.
    
    def newStrokeAdded(self): #bind events for the newly added stroke item
        self.app.sf.addNewStrokeItem() #add new stroke item
        #bind events
        index = len(self.app.sf.strokeList)-1
        if(index != 0): #configure the group option so it is the same group as the previous stroke
            self.app.sf.strokeList[index].groupChoice.set(self.app.sf.strokeList[index-1].groupChoice.get())
        self.app.sf.strokeList[index].strokeChoice.trace("w", lambda *args, ind=index, me=self: me.strokeOptionChange(ind))
        self.app.sf.strokeList[index].groupChoice.trace("w", lambda *args, ind=index, me=self: me.strokeGroupChange(ind))
        self.app.sf.strokeList[index].deleteButton.configure(command = lambda ind=index, me=self: me.strokeDelete(ind))
        self.app.sf.strokeList[index].detectButton.configure(command = lambda ind=index, me=self: me.strokeDetect(ind)) 
        self.app.sf.strokeList[index].editButton.configure(command = lambda ind=index, me=self: me.strokeEdit(ind))
        
        #recolor all the strokes evenly with hsv
        self.app.sf.adjustColoring()
    
    #stroke events, more difficult to work with
    def strokeOptionChange(self, ind):
        val = self.app.sf.strokeList[ind].strokeChoice.get()[2:] #skip the symbol and space characters, just get the name
        self.fitData.fits[ind].name = val
        self.app.pf.fitCopy.fits[ind].name = val
        self.app.pf.fitCopy.fits[ind]
        self.app.pf.redrawStroke(ind, self.app.pf.edit == ind) #second arg is edit mode
    def strokeGroupChange(self, ind): #this probably shouldn't do anything, right?
        return
        #val = self.app.sf.strokeList[ind].strokeChoice.get()
        #print("change group", ind, val)
    def strokeDelete(self, ind):
        print("delete stroke", ind)
        self.app.sf.strokeList[ind].frame.destroy() #remove this stroke item from the ui
        self.app.sf.strokeList[ind].deleted = True
        self.app.sf.frame.update() #forces scrollbar to update
        self.app.sf.onFrameConfigure(0) #allows scrollbar to be updated
        
    def strokeDetect(self, ind):
        print("detect stroke", ind)
    def strokeEdit(self, ind):
        if(not self.app.sf.strokeList[ind].editMode): #change to editing
            #disable edit on all other buttons
            for i in range(len(self.app.sf.strokeList)):
                self.app.sf.strokeList[i].stopEdit()
                self.app.pf.editStroke(i, False)
            self.app.sf.strokeList[ind].startEdit()
            self.app.pf.editStroke(ind, True)
            self.app.pf.edit = ind
        else: #turn off editing.
            self.app.sf.strokeList[ind].stopEdit()
            self.app.pf.editStroke(ind, False)
            self.app.pf.edit = -1 #no stroke is being edited
        #UPDATE the canvas 
    
    #need at least these events
    #charCode change
    #charSymbol change

    #auto-gen
    #load
    #save
    #save & export

    #canvas click/drag/move

    #for each stroke
    #type change
    #group change
    #delete (?)
    #detect
    #edit
        
app = App()
app.run()